In [ ]:
import pandas as pd
import numpy as np

In [ ]:
customer_df = pd.read_csv('/Users/jacob.perius/psa_segment_testing/11_4_24_CustomerData_20241104_000000000000.csv', low_memory=False)

customer_df

In [ ]:
print(customer_df.columns)

customer_df = customer_df[customer_df['latest_order_date'] > '2024-01-01']

customer_df 

In [ ]:
customer_df['segment'].unique()

In [ ]:
customer_df['submission_count'].head()

In [ ]:
import re

customer_df.loc[:, 'zip_code'] = customer_df.loc[:, 'zip_code'].str.split('-').str[0]
customer_df.loc[:, 'zip_code'] = customer_df.loc[:, 'zip_code'].apply(lambda x: np.nan if x == 'nan' else x)

customer_df = customer_df[customer_df['zip_code'].notna() & (customer_df['zip_code'] != '')]

customer_df = customer_df[~customer_df['zip_code'].str.contains('@')]
customer_df.loc[:, 'zip_code'] = customer_df.loc[:, 'zip_code'].str.strip("'")
customer_df.loc[:, 'zip_code'] = customer_df.loc[:, 'zip_code'].apply(lambda x: re.sub(r'\D', '', str(x)))
customer_df = customer_df[customer_df['zip_code'] != '']

In [ ]:
customer_df['zip_code'] = customer_df['zip_code'].str.zfill(5)

In [ ]:
dma_df = pd.read_csv('/Users/jacob.perius/psa_segment_testing/ENV _ Census _ ZIP to DMA.csv')

dma_df.head(5)

In [ ]:
dma_df['zip_code'] = dma_df['zip_code'].astype('str').str.zfill(5)

In [ ]:
merged_df = pd.merge(customer_df, dma_df, on='zip_code', how='left')

merged_df.drop(columns=['date_updated_at'], inplace=True)

merged_df


In [ ]:
group_counts_df = merged_df.groupby(['dma_code', 'dma_description']).size().reset_index(name='count')

group_counts_df

In [ ]:
grouped_df = merged_df.groupby(['dma_code', 'dma_description']).agg({
    'submission_count': 'sum',
    'submission_total_qty': 'sum',
    'submission_total_dv': 'sum',
    'avg_dv_per_sub': 'mean',
    'avg_item_per_sub': 'mean',
    'total_orders': 'sum',
    'total_qty_ordered': 'sum',
    'total_order_revenue': 'sum',
    'submission_count_2023': 'sum',
    'submission_total_qty_2023': 'sum',
    'submission_total_dv_2023': 'sum',
    'total_orders_2023': 'sum',
    'total_qty_ordered_2023': 'sum',
    'total_order_revenue_2023': 'sum'
}).reset_index()

grouped_df

In [ ]:
final_df = pd.merge(grouped_df, group_counts_df, on=['dma_code', 'dma_description'], how='left')

final_df

In [ ]:
population_df = pd.read_csv('/Users/jacob.perius/psa_segment_testing/zip_grouped_census_df.csv')

population_df.head()

In [ ]:
final_df = pd.merge(final_df, population_df, on='dma_code', how='left')

final_df

In [ ]:
old_cols = [
    col for col in final_df.columns
    if col not in ['dma_code', 'dma_description', 'population'] 
    and not any(keyword in col for keyword in ['normalized', 'avg'])
]
new_cols = [f'population_normalized_{col}' for col in old_cols]

final_df[new_cols] = final_df[old_cols].div(final_df['population'], axis=0)

final_df

In [ ]:
old_cols2 = [
    col for col in final_df.columns
    if col not in ['dma_code', 'dma_description', 'population', 'count'] 
    and not any(keyword in col for keyword in ['normalized', 'avg'])
]
new_cols2 = [f'intra_dma_avg_{col}' for col in old_cols2]

final_df[new_cols2] = final_df[old_cols2].div(final_df['count'], axis=0)


In [ ]:
old_cols3 = [
    col for col in final_df.columns
    if col not in ['dma_code', 'dma_description', 'population'] 
    and not any(keyword in col for keyword in ['normalized', 'avg'])
]
new_cols3 = [f'percent_of_total_{col}' for col in old_cols3]

final_df[new_cols3] = final_df[old_cols3].apply(lambda x: x / x.sum(), axis=0)

In [ ]:
final_df = final_df.drop(columns=old_cols2)

final_df

In [ ]:
import re

google_data_df = pd.read_csv('/Users/jacob.perius/psa_segment_testing/google_ads_data.csv')
google_data_df = google_data_df.iloc[:-6]

google_data_df['zip_code'] = google_data_df['Matched location'].apply(lambda x: re.search(r'^\d{5}', x).group(0))
google_data_df

In [ ]:
google_data_df['Impr.'] = google_data_df['Impr.'].str.replace(',', '').str.strip().astype('int64')
google_data_df['Clicks'] = google_data_df['Clicks'].str.replace(',', '').str.strip().astype('int64')
google_data_df['cvr'] = google_data_df['Clicks'] / google_data_df['Impr.']
google_data_df

In [ ]:
google_dma_df = pd.merge(google_data_df, dma_df, on='zip_code', how='right')[['dma_code', 'dma_description', 'cvr']]

google_dma_df

In [ ]:
grouped_google_dma_df = google_dma_df.groupby(['dma_code', 'dma_description']).mean().reset_index()

grouped_google_dma_df

In [ ]:
final_df_with_cvr = pd.merge(final_df, grouped_google_dma_df, on=['dma_code', 'dma_description'], how='left')

final_df_with_cvr

In [ ]:
#features = final_df_with_cvr.iloc[:, 2:]

relevant_features = [
    feature for feature in final_df_with_cvr.iloc[:, 2:].columns
    if (('intra' in feature) or (feature =='cvr')) and ('dv' not in feature) #if ((feature != 'count') and (feature != 'population')
]

#features = features.drop(columns=['population']) #trying to focus on features that are not population or percent of total
features = final_df_with_cvr[relevant_features]

features

In [ ]:
features.columns

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(features)

x_scaled_df =pd.DataFrame(X_scaled)

In [ ]:
x_scaled_df.describe()

In [ ]:
# min_dist (e.g., to 0.2 or 0.3) might encourage more spread and separation between clusters.

from umap import UMAP

umap = UMAP(n_components=2, random_state=42, metric='euclidean', n_neighbors=15, min_dist=0.1) #25

X_umap = umap.fit_transform(X_scaled)

X_pca_df2 = pd.DataFrame(X_umap)

In [ ]:
X_pca_df2.rename(columns={0: 'x', 1: 'y', 2: 'z'}, inplace=True)
X_pca_df2

In [ ]:
from sklearn.cluster import KMeans, AgglomerativeClustering

# Perform clustering on UMAP embeddings
kmeans = KMeans(n_clusters=3, random_state=42)
cluster_labels = kmeans.fit_predict(X_umap)

#agglo = AgglomerativeClustering(n_clusters=3, linkage='ward')
#cluster_labels = agglo.fit_predict(X_umap)

final_df_with_cvr['cluster'] = cluster_labels
final_df_with_cvr['x'], final_df_with_cvr['y'] = X_pca_df2['x'], X_pca_df2['y']

final_df_with_cvr

In [ ]:
from scipy.spatial.distance import euclidean

centroids = final_df_with_cvr.groupby('cluster')[['x', 'y']].mean()

# Calculate the distance of each point to its own cluster's centroid
def calculate_distance(row):
    cluster_id = row['cluster']
    centroid = centroids.loc[cluster_id]
    return euclidean((row['x'], row['y']), centroid)

final_df_with_cvr['distance_to_centroid'] = final_df_with_cvr.apply(calculate_distance, axis=1)

group_assignments = []

for cluster_id, group_df in final_df_with_cvr.groupby('cluster'):
    # Sort points within the cluster by distance to the centroid
    sorted_df = group_df.sort_values(by='distance_to_centroid')
    
    # Calculate group sizes
    count = len(sorted_df)
    group_size = count // 3
    remainder = count % 3
    
    # Assign groups based on distance ranking within each cluster
    group_labels = ['Group A'] * group_size + ['Group B'] * group_size + ['Group C'] * group_size

    # Handle any remainders
    for i in range(remainder):
        group_labels.append(f'Group {chr(65 + i)}')
    
    sorted_df['group'] = group_labels
    
    group_assignments.append(sorted_df)

final_df_with_cvr = pd.concat(group_assignments).sort_index()

final_df_with_cvr

In [ ]:
'''
import plotly.express as px

fig = px.scatter_3d(X_pca_df2, x='x', y='y', z='z', 
                    color=final_df_with_cvr['cluster'], 
                    title='DMAs Clustered by UMAP & K-Means',
                    opacity=0.7,
                    hover_name=final_df_with_cvr['dma_description'],
                    labels={'color': 'Cluster'}
                    )

fig2 = px.scatter_3d(X_pca_df2, x='x', y='y', z='z', 
                    color=final_df_with_cvr['group'], 
                    title='DMAs Stratified by Group',
                    opacity=0.7,
                    hover_name=final_df_with_cvr['dma_description'],
                    labels={'color': 'Experiment Group'}
                    )

fig.show()
fig2.show()
'''

In [ ]:

import plotly.express as px

fig = px.scatter(X_pca_df2, x='x', y='y', color=final_df_with_cvr['cluster'].astype(str))

fig.show()

In [ ]:
from dotenv import load_dotenv
import matplotlib.pyplot as plt
import seaborn as sns
import arviz as az

load_dotenv()

import pymc as pm

#features_of_interest = final_df_with_cvr.drop(columns=['population', 'count']).columns[3:-2]

for feature in relevant_features: #features_of_interest:

    data_a = final_df_with_cvr[final_df_with_cvr['group'] == 'Group A'][feature]
    data_b = final_df_with_cvr[final_df_with_cvr['group'] == 'Group B'][feature]
    data_c = final_df_with_cvr[final_df_with_cvr['group'] == 'Group C'][feature]

    with pm.Model() as model:

        # Priors for group means and standard deviations
        #mu_a = pm.Normal("mu_a", mu=5, sigma=1)
        #mu_b = pm.Normal("mu_b", mu=5, sigma=1)
        #mu_c = pm.Normal("mu_c", mu=5, sigma=1)

        mu_a = pm.Normal("mu_a", mu=0, sigma=100)
        mu_b = pm.Normal("mu_b", mu=0, sigma=100)
        mu_c = pm.Normal("mu_c", mu=0, sigma=100)

        sigma_a = pm.HalfNormal("sigma_a", sigma=1)
        sigma_b = pm.HalfNormal("sigma_b", sigma=1)
        sigma_c = pm.HalfNormal("sigma_c", sigma=1)

        # Likelihoods for observed data
        obs_a = pm.Normal("obs_a", mu=mu_a, sigma=sigma_a, observed=data_a)
        obs_b = pm.Normal("obs_b", mu=mu_b, sigma=sigma_b, observed=data_b)
        obs_c = pm.Normal("obs_c", mu=mu_c, sigma=sigma_c, observed=data_c)

        # Sampling
        trace = pm.sample(1000, chains=4)

    print(trace.posterior)

    # Check Posterior Overlap
    #pm.plot_posterior(trace, var_names=["mu_a", "mu_b", "mu_c"])

    mu_a_samples = trace.posterior['mu_a'].values.flatten()
    mu_b_samples = trace.posterior['mu_b'].values.flatten()
    mu_c_samples = trace.posterior['mu_c'].values.flatten()

    #hdi_mu_a = az.hdi(mu_a_samples, hdi_prob=0.94)
    #hdi_mu_b = az.hdi(mu_b_samples, hdi_prob=0.94)
    #hdi_mu_c = az.hdi(mu_c_samples, hdi_prob=0.94)

    plt.figure(figsize=(10, 6))

    # Plot KDE for each variable with different colors
    sns.kdeplot(mu_a_samples, fill=True, alpha=0.5, label='Group A')
    sns.kdeplot(mu_b_samples, fill=True, alpha=0.5, label='Group B')
    sns.kdeplot(mu_c_samples, fill=True, alpha=0.5, label='Group C')

    plt.xlabel(f"{feature}")
    plt.ylabel("Density")
    plt.title("Overlayed Posterior Distributions for Experiment Groups A, B & C")
    plt.legend()

    plt.show()

    #print(f"Group A {feature} HDI: {hdi_mu_a}")
    #print(f"Group B {feature} HDI: {hdi_mu_b}")
    #print(f"Group C {feature} HDI: {hdi_mu_c}")

In [ ]:
dmas_a = final_df_with_cvr[final_df_with_cvr['group'] == 'Group A']

dmas_a.sort_values(by='percent_of_total_submission_count', ascending=False).reset_index(drop=True)

In [ ]:
dmas_b = final_df_with_cvr[final_df_with_cvr['group'] == 'Group B']

dmas_b.sort_values(by='percent_of_total_submission_count', ascending=False)

In [ ]:
dmas_c = final_df_with_cvr[final_df_with_cvr['group'] == 'Group C']

dmas_c.sort_values(by='percent_of_total_submission_count', ascending=False)

In [ ]:
names_and_groups_df = pd.concat([dmas_a, dmas_b, dmas_c], axis=0) #[['group', 'dma_code', 'dma_description']]
#names_and_groups_df['dma_code'] = names_and_groups_df['dma_code'].astype('int64')
names_and_groups_df['group'] = names_and_groups_df['group'].astype('category')
names_and_groups_df

In [ ]:
#names_and_groups_df.to_csv('names_and_groups.csv', index=False)

In [ ]:
import json
import plotly.express as px

with open('/Users/jacob.perius/psa_segment_testing/neilsen-dma-markets-albers-projection_1356.geojson', 'r') as f:
    dma_geojson_str = f.read()

# First parse to remove the outer string layer
dma_geojson = json.loads(dma_geojson_str)

# Second parse if needed (in case it's double-encoded)
if isinstance(dma_geojson, str):
    dma_geojson = json.loads(dma_geojson)

fig = px.choropleth(
    names_and_groups_df,
    geojson=dma_geojson,
    locations='dma_code',
    color='group',
    featureidkey='properties.dma_code',
    color_discrete_map={
        'Group A': 'blue',
        'Group B': 'orange',
        'Group C': 'green'
    },
    hover_data={'dma_code': True, 'dma_description': True, 'group': True}
)

fig.update_traces(marker_line_width=1, marker_opacity=1.0)

fig.update_geos(
    fitbounds="locations",
    visible=False,
)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0},
                legend_title_text='Test Group'
                )

fig.show()

In [ ]:
fig.write_html('test.html')

In [ ]:
grouped_customers_df = pd.merge(merged_df, names_and_groups_df, on=['dma_code', 'dma_description'], how='left')

grouped_customers_df